In [219]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.keys import Keys
boxer_list = ['Deontay Wilder']
def get_boxers(boxers):
    page_link = 'http://beta.compuboxdata.com/fighter'
    chromedriver = 'C:\\Users\\User\\Downloads\\chromedriver'
    cdriver = webdriver.Chrome(chromedriver)
    cdriver.maximize_window()
    cdriver.get(page_link)
    wait = WebDriverWait(cdriver,10)
    wait.until(EC.visibility_of_element_located((By.ID,'s2id_autogen1'))).send_keys(str(boxers))
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'select2-result-label'))).click()
    while True:
        try:
            element = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'view_more')))
            cdriver.execute_script("arguments[0].click();",element)
        except TimeoutException:
            break
    fighters =  cdriver.find_elements_by_xpath("//div[@class='row row-bottom-margin-5']/div[2]")
    cols = ['fighters', 'stats']
    fight_data = pd.DataFrame(columns = cols)
    for fighter in fighters:
        fight_data.append({'fighters':fighter.text},ignore_index=True)
        cdriver.find_element_by_xpath("//a[contains(@onclick,'get_fight_report')]").click()
        punches = cdriver.find_elements_by_xpath("//div[@class='modal-content']")
        for punch in punches:
            fight_data.append({'stats':punch.text},ignore_index=True)
    cdriver.refresh()
    return fight_data
boxing_dict = {boxer.replace(' ',''):pd.DataFrame([get_boxers(boxer)]) for boxer in boxer_list}


In [236]:
#testing on one boxer
page_link = 'http://beta.compuboxdata.com/fighter'
chromedriver = 'C:\\Users\\User\\Downloads\\chromedriver'
cdriver = webdriver.Chrome(chromedriver)
cdriver.maximize_window()
cdriver.get(page_link)
wait = WebDriverWait(cdriver,20)
wait.until(EC.visibility_of_element_located((By.ID,'s2id_autogen1'))).send_keys('Deontay Wilder')
wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'select2-result-label'))).click()
while True:
    try:
        element = cdriver.find_element_by_class_name('view_more')
        webdriver.ActionChains(cdriver).move_to_element(element).click(element).perform()
#         element = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'view_more'))).click()
    except TimeoutException:
        break
# fighters =  cdriver.find_elements_by_xpath("//div[@class='row row-bottom-margin-5']/div[2]")

links = [x.get_attribute('onclick') for x in wait.until(EC.visibility_of_element_located((By.XPATH, "//*[contains(@onclick, 'get_fight_report')]/a")))]
htmls = []
for link in links:
    cdriver.get(link)
    htmls.append(cddriver.page_source)
# for fighter in fighters:
#     print(fighter.text)
#     fight_data.append({'fighters':fighter.text},ignore_index=True)
#     elem = wait.until(EC.element_to_be_clickable((By.XPATH,"//a[contains(@onclick,'get_fight_report')]")))
#     elem.click()
#     punches = cdriver.find_elements_by_xpath("//div[@class='modal-content']")
#     for punch in punches:
#         fight_data.append({'stats':punch.text},ignore_index=True)

KeyboardInterrupt: 

In [240]:
boxer_list = ['Deontay Wilder','Tyson Fury','Andy Ruiz']
def get_boxers(boxers):
    page_link = 'http://beta.compuboxdata.com/fighter'
    chromedriver = 'C:\\Users\\User\\Downloads\\chromedriver'
    cdriver = webdriver.Chrome(chromedriver)
    cdriver.maximize_window()
    cdriver.get(page_link)
    wait = WebDriverWait(cdriver,10)
    wait.until(EC.visibility_of_element_located((By.ID,'s2id_autogen1'))).send_keys(str(boxers))
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'select2-result-label'))).click()
    while True:
        try:
            element = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'view_more')))
            element.click()
        except :
            break
    fighters =  cdriver.find_elements_by_xpath("//div[@class='row row-bottom-margin-5']/div[2]")
    cols = ['fighters', 'stats']
    fight_data = pd.DataFrame(columns = cols)
    for fighter in fighters:
        fight_data=fight_data.append({'fighters':fighter.text},ignore_index=True)
    links = [x.get_attribute('onclick') for x in wait.until(EC.visibility_of_element_located((By.XPATH, "//*[contains(@onclick, 'get_fight_report')]/a")))]
    htmls = []
    for link in links:
        cdriver.get(link)
        htmls.append(cddriver.page_source)
    cdriver.refresh()
    return htmls
get_boxers('Deontay Wilder')
# # boxing_dict = {boxer.replace(' ',''):pd.DataFrame([get_boxers(boxer)]) for boxer in boxer_list}
#         cdriver.find_element_by_xpath("//a[contains(@onclick,'get_fight_report')]").click()
#         punches = cdriver.find_elements_by_xpath("//div[@class='modal-content']")
#         for punch in punches:
#             fight_data=fight_data.append({'stats':punch.text},ignore_index=True)


TimeoutException: Message: 


In [238]:
boxing_dict

{'DeontayWilder':                                                    0
 0                                fighters stats
 0..., 'TysonFury':                                                    0
 0                            fighters stats
 0   F..., 'AndyRuiz':                                                    0
 0                               fighters stats
 0 ...}